In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime

In [2]:
"""
Input: p10 combined pba40 plu and basis boc data, with 
Output: p10 combined pba40 plu and basis boc data, with: 
    1) NaN in basis allowed development types filled in using pba40 data
    2) basis allowed development types replaced by pba40 data following the 'hybrid index'
"""

if os.getenv('USERNAME')    =='ywang':
    BOX_dir                 = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim 1.5\\PBA50'.format(os.getenv('USERNAME'))
    GitHub_petrale_dir      = 'C:\\Users\\{}\\Documents\\GitHub\\petrale\\'.format(os.getenv('USERNAME'))
    
# input file locations
hybrid_index_dir        = os.path.join(GitHub_petrale_dir, 'policies\\plu\\base_zoning\\hybrid_index')
pba40_zoning_box_dir    = os.path.join(BOX_dir, 'OLD PBA50 Large General Input Data')
pba50_zoningmod_dir     = os.path.join(BOX_dir, 'Policies\\Zoning Modifications')
raw_plu_boc_dir         = os.path.join(BOX_dir, 'Policies\\Base zoning\\outputs')
other_inputs_dir        = os.path.join(BOX_dir, 'Policies\\Base zoning\\inputs')

# output file location
data_output_dir         = os.path.join(BOX_dir, 'Policies\\Base zoning\\outputs\\hybrid_base_zoning')


ALLOWED_BUILDING_TYPE_CODES = ["HS","HT","HM","OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]
RES_BUILDING_TYPE_CODES     = ["HS","HT","HM",                                        "MR"          ]
NONRES_BUILDING_TYPE_CODES  = [               "OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]

today = datetime.today().strftime('%Y_%m_%d')

In [3]:
## P10 parcels with pba40 plu and basis boc data
plu_boc_file = os.path.join(raw_plu_boc_dir, today+'_p10_plu_boc_allAttrs.csv')
plu_boc = pd.read_csv(plu_boc_file)
print("Read {:,} rows from {}".format(len(plu_boc), plu_boc_file))
display(plu_boc.head())

Read 1,956,208 rows from C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim 1.5\PBA50\Policies\Base zoning\outputs\2020_05_06_p10_plu_boc_allAttrs.csv


,PARCEL_ID,county_id,county_name,juris_zmod,ACRES,zoning_id_pba40,pba50zoningmodcat_zmod,max_far_basis,max_far_pba40,source_far_basis,...,RS_basis,RS_pba40,RB_basis,RB_pba40,MR_basis,MR_pba40,MT_basis,MT_pba40,ME_basis,ME_pba40
0,229116.0,1,Alameda,livermore,3.360520,60126.0,livermoreNANAHRADRNAinNA,0.00,NaN,basis,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166.0,1,Alameda,livermore,1.294423,11903.0,livermoreNANADRNAinNA,0.35,NaN,basis,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378.0,1,Alameda,hayward,14.993605,11803.0,haywardNANANANAinNA,0.00,1.363636,basis,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2004420.0,97,Sonoma,unincorporated_sonoma,316.247146,12975.0,unincorporated_sonomaNANADRNAoutNA,0.00,1.590909,basis,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,340332.0,1,Alameda,fremont,0.621275,2511.0,fremontNANAHRADRNAinNA,0.01,2.363636,basis,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [4]:
## Index 0: fill in missing allowed development types in BASIS

def countMissing(df):
    for btype in ALLOWED_BUILDING_TYPE_CODES:
        null_btype_count = len(df.loc[df["{}_basis".format(btype)].isnull()])
        print('Number of parcels missing allowable type for {}: {:,} or {:.1f}%'.format(btype,
           null_btype_count, 100.0*null_btype_count/len(df)))

print('Count number of parcels missing allowable development types in the BASIS data:')
countMissing(plu_boc)

plu_boc_filled_devTypeNa = plu_boc.copy()

for btype in ALLOWED_BUILDING_TYPE_CODES:
    plu_boc_filled_devTypeNa[btype+'_idx'] = 1
    missing_idx = (plu_boc_filled_devTypeNa[btype+'_basis'].isnull()) & (plu_boc_filled_devTypeNa['nodev_zmod'] == 0)
    plu_boc_filled_devTypeNa.loc[missing_idx, btype+'_basis'] = plu_boc_filled_devTypeNa.loc[missing_idx, btype+'_pba40']
    plu_boc_filled_devTypeNa.loc[missing_idx, btype+'_idx'] = 'PBA40_fill_na'

print('\n After filling nan in BASIS allowable development types using PBA40 data:')
countMissing(plu_boc_filled_devTypeNa)

#plu_boc_filled_devTypeNa.to_csv(os.path.join(data_output_dir, today+'_p10_plu_boc_fill_naType.csv'),index = False)

print('Print unique data source index for allowable development types (should only be "PBA40_fill_na" or "1"):')
for i in ALLOWED_BUILDING_TYPE_CODES:
    print('{}: {}'.format(i,plu_boc_filled_devTypeNa[i+'_idx'].unique()))

Count number of parcels missing allowable development types in the BASIS data:
Number of parcels missing allowable type for HS: 227,036 or 11.6%
Number of parcels missing allowable type for HT: 227,051 or 11.6%
Number of parcels missing allowable type for HM: 233,111 or 11.9%
Number of parcels missing allowable type for OF: 233,011 or 11.9%
Number of parcels missing allowable type for HO: 233,111 or 11.9%
Number of parcels missing allowable type for SC: 233,085 or 11.9%
Number of parcels missing allowable type for IL: 227,839 or 11.6%
Number of parcels missing allowable type for IW: 233,093 or 11.9%
Number of parcels missing allowable type for IH: 233,054 or 11.9%
Number of parcels missing allowable type for RS: 233,111 or 11.9%
Number of parcels missing allowable type for RB: 233,607 or 11.9%
Number of parcels missing allowable type for MR: 233,111 or 11.9%
Number of parcels missing allowable type for MT: 233,111 or 11.9%
Number of parcels missing allowable type for ME: 233,173 or 11.

In [5]:
## Assign allow residential and/or non-residential by summing the columns
## for the residential/nonresidential allowed building type codes
## Returns dataframe with PARCEL_ID, allow_res_[boc_source], allow_nonres_[boc_source]

def set_allow_dev_type(df_original,boc_source):
    # don't modify passed df
    df = df_original.copy()

    # note that they can't be null because then they won't sum -- so make a copy and fillna with 0
    for dev_type in ALLOWED_BUILDING_TYPE_CODES:
        df[dev_type+"_"+boc_source] = df[dev_type+"_"+boc_source].fillna(value=0.0)    
    
    # allow_res is sum of allowed building types that are residential
    res_allowed_columns = [btype+'_'+boc_source for btype in RES_BUILDING_TYPE_CODES]
    df['allow_res_' +boc_source] = df[res_allowed_columns].sum(axis=1)
    
    # allow_nonres is the sum of allowed building types that are non-residential
    nonres_allowed_columns = [btype+'_'+boc_source for btype in NONRES_BUILDING_TYPE_CODES]
    df['allow_nonres_'+boc_source] = df[nonres_allowed_columns].sum(axis=1)
    
    return df[['PARCEL_ID',
               "allow_res_"    +boc_source,
               "allow_nonres_" +boc_source]]

In [6]:
## Apply hybrid index to raw plu_boc data. 
## Returns plu_boc with updated allowable dev type and intensity parameters for BASIS fields, 
## along with '_idx' for each parameter to indicate the data source - '1' for BASIS '0' for PBA40

def apply_hybrid_idx(df_origional,hybrid_idx):
    # don't modify passed df
    df = df_origional.copy()
    
    for juris in juris_list:
        
        print('')
        print('Apply hybrid index for: {}'.format(juris))

        for devType in ALLOWED_BUILDING_TYPE_CODES:
            df[btype+'_idx'] = 1
            if hybrid_idx[devType+'_idx'][juris] == 0:
                print('Use PBA40 data for {} -'.format(devType))
                #print('Before applying the index, parcel counts by data source for {}:'.format(devType))
                #display(df.loc[df.juris_zmod == juris][devType+'_idx'].value_counts())
                
                replace_idx = (df.juris_zmod == juris) & (df[devType+'_idx'] != 'PBA40_fill_na')
                df.loc[replace_idx, devType+'_basis'] = df.loc[replace_idx, devType+'_pba40']
                df.loc[replace_idx, devType+'_idx'] = 0
                #print('After applying the index, parcel counts by data source for {}:'.format(devType))
                #display(df.loc[df.juris_zmod == juris][devType+'_idx'].value_counts())   
                
            elif hybrid_idx[devType+'_idx'][juris] == 1:
                print('Use BASIS data for {}'.format(devType))
        

        for intensity in ['max_dua','max_far','max_height']:
            df[intensity+'_idx'] = 1
            if hybrid_idx[intensity+'_idx'][juris] == 0:
                print('Use PBA40 data for {}'.format(intensity))
                #print('Before applying the index, parcel counts by data source for {}:'.format(intensity))
                #display(df.loc[df.juris_zmod == juris][intensity+'_idx'].value_counts())
                
                replace_idx = df.juris_zmod == juris
                df.loc[replace_idx, intensity+'_basis'] = df.loc[replace_idx, intensity+'_pba40']
                df.loc[replace_idx, intensity+'_idx'] = 0
                #print('After applying the index, parcel counts by data source for {}:'.format(intensity))
                #display(df.loc[df.juris_zmod == juris][intensity+'_idx'].value_counts())
                
            elif hybrid_idx[intensity+'_idx'][juris] == 1:
                print('Use BASIS data for {}'.format(intensity))
                print('')

    return df

In [48]:
## Apply the hybrid index of each hybrid version

for hybrid_idx_file in list(glob.glob(hybrid_index_dir+'/*.csv')):
    hybrid_name = os.path.basename(hybrid_idx_file).split('.')[0][4:]
    print('Hybrid version: {}'.format(hybrid_name))
    hybrid_idx = pd.read_csv(hybrid_idx_file)
    hybrid_idx.rename(columns = {'MAX_FAR_idx'   : 'max_far_idx', 
                                 'MAX_DUA_idx'   : 'max_dua_idx',
                                 'MAX_HEIGHT_idx': 'max_height_idx'}, inplace = True)
    display(hybrid_idx.head())
    hybrid_idx.set_index('juris_name',inplace = True)
    juris_list =list(hybrid_idx.index.values)
    
    for devType in ALLOWED_BUILDING_TYPE_CODES:
        print(devType)
        display(plu_boc_filled_devTypeNa[devType+'_idx'].value_counts())
          
    plu_boc_hybrid = apply_hybrid_idx(plu_boc_filled_devTypeNa,hybrid_idx)
    
    for devType in ALLOWED_BUILDING_TYPE_CODES:
        print(devType)
        display(plu_boc_hybrid[devType+'_idx'].value_counts())
        
    for intensity in ['max_far','max_far','max_height']:
        print(intensity)
        display(plu_boc_hybrid[intensity+'_idx'].value_counts())
        
    # recalculate 'allow_res' and 'allow_nonres' based on the allowable development type
    allowed_basis = set_allow_dev_type(plu_boc_hybrid,'basis')
    allowed_pba40 = set_allow_dev_type(plu_boc_hybrid,'pba40')
    
    # drop the previous 'allow_res' and 'allow_nonres' and insert the new ones
    plu_boc_hybrid.drop(columns = ['allow_res_basis', 'allow_nonres_basis', 
                                   'allow_res_pba40', 'allow_nonres_pba40'], inplace = True)
    plu_boc_hybrid = plu_boc_hybrid.merge(allowed_basis, 
                                          on = 'PARCEL_ID', 
                                          how = 'left').merge(allowed_pba40, 
                                                              on = 'PARCEL_ID', 
                                                              how = 'left')
  
    #plu_boc_hybrid.to_csv(os.path.join(data_output_dir, today+'_p10_plu_boc_'+hybrid_name+'.csv'),index = False)
    

Hybrid version: BASIS_devType_intensity_partial


,juris_id,juris_name,county,OF_idx,HO_idx,SC_idx,IL_idx,IW_idx,IH_idx,RS_idx,...,ME_idx,HS_idx,HT_idx,HM_idx,max_dua_idx,max_far_idx,max_height_idx,proportion_adj_dua,proportion_adj_far,proportion_adj_height
0,alam,alameda,ala,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,1,1,1
1,alba,albany,ala,1,0,0,0,0,0,1,...,0,0,0,1,1,1,1,1,1,1
2,berk,berkeley,ala,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,1
3,dubl,dublin,ala,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,1
4,emer,emeryville,ala,1,0,0,0,0,0,1,...,0,0,0,1,1,1,1,1,1,1


HS


1                1749712
PBA40_fill_na     206496
Name: HS_idx, dtype: int64

HT


...

1    1947672
0       8536
Name: max_far_idx, dtype: int64

max_far


1    1947672
0       8536
Name: max_far_idx, dtype: int64

max_height


1    1947672
0       8536
Name: max_height_idx, dtype: int64

In [8]:
## Export zoning file as BAUS input
print(list(plu_boc_hybrid))

['PARCEL_ID', 'county_id', 'county_name', 'juris_zmod', 'ACRES', 'zoning_id_pba40', 'pba50zoningmodcat_zmod', 'max_far_basis', 'max_far_pba40', 'source_far_basis', 'source_far_pba40', 'max_dua_basis', 'max_dua_pba40', 'source_dua_basis', 'source_dua_pba40', 'max_height_basis', 'max_height_pba40', 'nodev_zmod', 'nodev_pba40', 'building_types_source_basis', 'source_basis', 'plu_id_basis', 'plu_jurisdiction_basis', 'plu_description_basis', 'HS_basis', 'HS_pba40', 'HT_basis', 'HT_pba40', 'HM_basis', 'HM_pba40', 'OF_basis', 'OF_pba40', 'HO_basis', 'HO_pba40', 'SC_basis', 'SC_pba40', 'IL_basis', 'IL_pba40', 'IW_basis', 'IW_pba40', 'IH_basis', 'IH_pba40', 'RS_basis', 'RS_pba40', 'RB_basis', 'RB_pba40', 'MR_basis', 'MR_pba40', 'MT_basis', 'MT_pba40', 'ME_basis', 'ME_pba40', 'HS_idx', 'HT_idx', 'HM_idx', 'OF_idx', 'HO_idx', 'SC_idx', 'IL_idx', 'IW_idx', 'IH_idx', 'RS_idx', 'RB_idx', 'MR_idx', 'MT_idx', 'ME_idx', 'max_dua_idx', 'max_far_idx', 'max_height_idx', 'allow_res_basis', 'allow_nonres_ba

In [50]:
# Parcels missing zoning_id_pba40 but have a plu_id_basis
test = plu_boc_hybrid.copy()

parcels_new_zone_id_idx = (test['zoning_id_pba40'].isnull()) & (test['plu_id_basis'].notnull())
parcels_new_zone_id = test.loc[parcels_new_zone_id_idx]

print('Number of parcels with plu_id_basis but no zoning_id_pba40: {}'.format(parcels_new_zone_id.shape[0]))

test.zoning_id_pba40 = test.zoning_id_pba40.fillna(-1).astype(np.int64)

new_plu_id = list(parcels_new_zone_id.plu_id_basis.unique())
print('Number of unique plu_id_basis from these parcels: {}'.format(len(new_plu_id)))

# the largest number of zoning_id_pba40 is 99021 except for '99999', therefore, the new zone code starts from the integer following 
# the second largest zoning_id_pba40
old_zone_id = sorted(list(test.zoning_id_pba40.unique()))
print('Number of unique zoning_id_pba40: {}'.format(len(old_zone_id)))
end_of_old_zone_id = old_zone_id[-2]
gap = old_zone_id[-1] - old_zone_id[-2] - 1

if gap > len(new_plu_id):
    new_zone_id = list(range(end_of_old_zone_id+1, end_of_old_zone_id+1+len(new_plu_id)))
    zone_id_lookup =dict(zip(new_plu_id + old_zone_id , new_zone_id + old_zone_id))
    
    test['zoning_id_pba50_temp'] = test['zoning_id_pba40']
    test.loc[parcels_new_zone_id_idx,'zoning_id_pba50_temp'] = test.loc[parcels_new_zone_id_idx,'plu_id_basis']
    test['zoning_id_pba50'] = test['zoning_id_pba50_temp'].map(zone_id_lookup)
    print('Number of unique zoning_id_pba50: {}'.format(len(test['zoning_id_pba50'].unique())))
    
else:
    print('Cannot generate new zoning_id_pba50 for all new plu_id_basis.')


Number of parcels with plu_id_basis but no zoning_id_pba40: (6167, 73)
Number of unique plu_id_basis from these parcels: 395
Number of unique zoning_id_pba40: 3241
Number of unique zoning_id_pba50: 3636


'print(\'Check if these parcels are all using BASIS zoning info:\')\nfor devType in ALLOWED_BUILDING_TYPE_CODES:\n    print(devType, ": \n")\n    print(parcels_new_zone_id[devType+\'_idx\'].value_counts())\nfor intensity in [\'max_dua\',\'max_far\',\'max_height\']:\n    print(intensity, ": \n")\n    print(parcels_new_zone_id[intensity+\'_idx\'].value_counts())'

In [51]:
test.loc[test.zoning_id_pba40 != test.zoning_id_pba50]

,PARCEL_ID,county_id,county_name,juris_zmod,ACRES,zoning_id_pba40,pba50zoningmodcat_zmod,max_far_basis,max_far_pba40,source_far_basis,...,ME_idx,max_dua_idx,max_far_idx,max_height_idx,allow_res_basis,allow_nonres_basis,allow_res_pba40,allow_nonres_pba40,zoning_id_pba50_temp,zoning_id_pba50
1185,580792.0,13,Contra Costa,concord,0.013816,-1,concordNANADRNAinNA,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,72ea781a-9e08-42a3-9e6c-0a21517d05b8,99022
1637,255073.0,1,Alameda,unincorporated_alameda,0.007685,-1,unincorporated_alamedaNAtra3NANAinunNA,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,514230c8-378d-4dda-837f-9a49dd675dc6,99023
1956,1220739.0,81,San Mateo,daly_city,0.059405,-1,daly_cityNANADRNAinNA,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,3721ff65-3b93-49dd-8d0c-980379fe2545,99024
1967,1455079.0,85,Santa Clara,unincorporated_santa_clara,0.157788,-1,unincorporated_santa_claraNANAHRANAoutsfd,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,27f53386-4369-482e-a327-bc859ea74531,99025
2357,1584312.0,85,Santa Clara,unincorporated_santa_clara,1.110776,-1,unincorporated_santa_claraNANAHRANAexp0sfd,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,ea989a9a-e6c9-461a-804c-d7575eac8fb1,99026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955103,650683.0,13,Contra Costa,pittsburg,0.114668,-1,pittsburgGGtra2NANAinsfd,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,9e49ab6a-653e-11e9-a923-1681be663d3e,99111
1955104,651123.0,13,Contra Costa,pittsburg,0.115417,-1,pittsburgGGtra2NANAinsfd,NaN,NaN,basis,...,1,1,1,1,0.0,0.0,0.0,0.0,9e49ab6a-653e-11e9-a923-1681be663d3e,99111
1955895,1411792.0,85,Santa Clara,san_jose,7.325086,-1,san_joseGGtra3DRNAinNA,NaN,NaN,missing,...,1,1,1,1,0.0,0.0,0.0,0.0,e2e7ea05-69ad-59c4-b764-435df3d391f3,99040
1956145,1414620.0,85,Santa Clara,san_jose,2.655408,-1,san_joseNAtra3HRANAinNA,NaN,NaN,missing,...,1,1,1,1,0.0,0.0,0.0,0.0,e2e7ea05-69ad-59c4-b764-435df3d391f3,99040


In [53]:
test[['plu_id_basis', 'plu_jurisdiction_basis', 'plu_description_basis']]

,plu_id_basis,plu_jurisdiction_basis,plu_description_basis
0,fc5f982a-40e3-452f-a660-f5f80b7d0b24,Livermore,Planned Development
1,4d5ddd2e-464a-4861-9c1f-88941a0b5676,Livermore,Residential
2,5a6d3363-b377-4431-8ac8-35aa1593225d,Hayward,Planned Development
3,0b967b71-e05b-46db-a689-4d3460b22887,Unincorporated Sonoma,Land Extensive Agriculture
4,9b689549-939f-4288-aad7-d3584fef122f,Fremont,Planned District
...,...,...,...
1956203,c8bd6a37-ef16-4746-8f32-267168d3252e,Antioch,Planned Development District
1956204,4120dc94-5d51-4ba4-8215-85208d84e70e,Concord,Single Family Residential
1956205,58f06623-a7ae-4410-aa1b-e14289dd1765,Novato,Business and Professional Office
1956206,16806a43-f6d9-4cff-84ec-14216e14a91e,Unincorporated Contra Costa,Planned Unit


In [57]:
zoning_parcels = test[['PARCEL_ID','zoning_id_pba50','plu_description_basis','juris_zmod','nodev_zmod']]
zoning_lookup = test[['zoning_id_pba50','juris_zmod','plu_description_basis'] + 
                     [intensity+'_basis' for intensity in ['max_far','max_far','max_height']] + 
                     [devType  +'_basis' for devType   in ALLOWED_BUILDING_TYPE_CODES]].drop_duplicates()

In [58]:
zoning_parcels.shape

(1956208, 5)

In [59]:
zoning_lookup.shape

(17468, 20)

In [60]:
len(plu_boc_hybrid.zoning_id_pba40.unique())

3241

In [63]:
test[[intensity+'_basis' for intensity in ['max_far','max_far','max_height']] + 
                     [devType  +'_basis' for devType   in ALLOWED_BUILDING_TYPE_CODES]].drop_duplicates()

,max_far_basis,max_far_basis,max_height_basis,HS_basis,HT_basis,HM_basis,OF_basis,HO_basis,SC_basis,IL_basis,IW_basis,IH_basis,RS_basis,RB_basis,MR_basis,MT_basis,ME_basis
0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.363636,1.363636,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.590909,1.590909,35.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.363636,2.363636,52.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940096,0.600000,0.600000,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1940322,0.000000,0.000000,30.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1941948,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
1945206,2.500000,2.500000,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
